# Deep Research: Sector Momentum Optimization

## Objective
Maximize Sharpe ratio for the Sector Momentum strategy.

## Strategy Overview
- **Assets**: 9 sector ETFs (XLK, XLF, XLV, XLY, XLP, XLE, XLB, XLU, XLRE)
- **Signal**: Dual momentum (relative strength + absolute momentum)
- **Rebalancing**: Monthly rotation to top-performing sectors
- **Risk Management**: VIX filter to skip rebalancing in high volatility

## Current Parameters
- `lookback_period`: 126 days (6 months)
- `vix_threshold`: 25 (skip rebalancing if VIX > 25)
- `leverage`: 1.5x

## Research Questions
1. What is the optimal lookback period for momentum calculation?
2. Should the VIX threshold be dynamic (percentile-based)?
3. What leverage maximizes risk-adjusted returns?
4. How many sectors should we hold (diversification vs concentration)?

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# Sector ETFs
SECTORS = {
    'XLK': 'Technology',
    'XLF': 'Financial',
    'XLV': 'Healthcare',
    'XLY': 'Consumer Discretionary',
    'XLP': 'Consumer Staples',
    'XLE': 'Energy',
    'XLB': 'Materials',
    'XLU': 'Utilities',
    'XLRE': 'Real Estate'
}

print(f"Sector Universe: {len(SECTORS)} ETFs")

In [ ]:
# Download data
print("Downloading sector data...")
sector_data = {}
for etf in SECTORS.keys():
    try:
        df = yf.download(etf, start="2010-01-01", end="2025-02-18", progress=False)
        if not df.empty:
            sector_data[etf] = df['Close'].copy()
            if isinstance(sector_data[etf], pd.DataFrame):
                sector_data[etf] = sector_data[etf].iloc[:, 0]
            print(f"  {etf}: {len(sector_data[etf])} days")
    except Exception as e:
        print(f"  {etf}: Error - {e}")

# Download VIX
vix = yf.download("^VIX", start="2010-01-01", end="2025-02-18", progress=False)
if isinstance(vix['Close'], pd.DataFrame):
    vix_close = vix['Close'].iloc[:, 0]
else:
    vix_close = vix['Close']

print(f"\nVIX: {len(vix_close)} days")

In [ ]:
# Analyze VIX for filter optimization
print("VIX Statistics:")
print(f"  Mean: {vix_close.mean():.2f}")
print(f"  Median: {vix_close.median():.2f}")
print(f"  Std: {vix_close.std():.2f}")
print(f"  75th percentile: {vix_close.quantile(0.75):.2f}")
print(f"  90th percentile: {vix_close.quantile(0.9):.2f}")

# Days above different thresholds
for threshold in [20, 25, 30]:
    high_vix_days = (vix_close > threshold).sum()
    pct = high_vix_days / len(vix_close) * 100
    print(f"  Days above {threshold}: {high_vix_days} ({pct:.1f}%)")

In [ ]:
# Calculate momentum signals
def calculate_momentum(prices, lookback=126):
    """
    Calculate momentum score for each sector.

    Momentum = (price / price_lookback_ago - 1) / volatility
    This is risk-adjusted momentum.
    """
    momentum = pd.DataFrame(index=prices[list(prices.keys())[0]].index)
    
    for etf, data in prices.items():
        # Total return over lookback period
        total_return = data.pct_change(lookback)
        
        # Volatility (annualized)
        vol = data.pct_change().rolling(lookback//2).std() * np.sqrt(252)
        
        # Risk-adjusted momentum
        momentum[etf] = total_return / vol
    
    return momentum

# Test with current parameters
momentum_126 = calculate_momentum(sector_data, lookback=126)
print("Momentum scores (latest date):")
print(momentum_126.iloc[-1].sort_values(ascending=False))

# Count how often each sector is top-performing
top_counts = momentum_126.idxmax(axis=1).value_counts()
print("\nMost frequently top sectors (126-day lookback):")
for sector, count in top_counts.items():
    print(f"  {SECTORS.get(sector, sector)}: {count} times")

In [ ]:
# Backtest sector momentum strategy
def backtest_sector_momentum(sector_data, vix_data,
                              lookback=126,
                              vix_threshold=25,
                              leverage=1.5,
                              top_n=3,
                              rebalance_freq='M'):
    """
    Backtest dual momentum sector rotation.
    
    - Select top_n sectors by risk-adjusted momentum
    - Skip rebalancing if VIX > threshold
    - Apply leverage to returns
    """
    # Create aligned dataframe
    prices = pd.DataFrame(sector_data)
    
    # Resample VIX to daily (if needed)
    if isinstance(vix_data, pd.Series):
        vix_daily = vix_data
    else:
        vix_daily = vix_data['Close']
        
    # Calculate returns
    returns = prices.pct_change()
    
    # Calculate momentum
    momentum = calculate_momentum(prices, lookback=lookback)
    
    # Rebalancing dates
    if rebalance_freq == 'M':
        rebalance_dates = pd.date_range(start=prices.index[0], end=prices.index[-1], freq='M')
    elif rebalance_freq == 'Q':
        rebalance_dates = pd.date_range(start=prices.index[0], end=prices.index[-1], freq='Q')
    else:
        rebalance_dates = pd.date_range(start=prices.index[0], end=prices.index[-1], freq='MS')
    
    # Track current holdings
    current_sectors = None
    sector_returns = pd.Series(0.0, index=prices.index)
    
    for i in range(1, len(prices)):
        current_date = prices.index[i]
        
        # Check if rebalancing day
        is_rebalance = any(rd.month == current_date.month and rd.year == current_date.year 
                           for rd in rebalance_dates)
        
        # Check VIX filter
        if is_rebalance:
            vix_val = vix_daily.loc[current_date] if current_date in vix_daily.index else vix_daily.iloc[-1]
            if vix_val > vix_threshold:
                # Skip rebalancing, keep current sectors
                pass
            else:
                # Select new top sectors
                if current_date in momentum.index:
                    mom = momentum.loc[current_date].dropna()
                    if len(mom) >= top_n:
                        current_sectors = mom.nlargest(top_n).index.tolist()
        
        # Calculate return for this day
        if current_sectors:
            # Equal weight among selected sectors
            daily_return = returns.loc[current_date, current_sectors].mean() * leverage
            sector_returns.iloc[i] = daily_return
    
    # Remove initial NaN
    sector_returns = sector_returns.iloc[lookback:]
    
    # Metrics
    sharpe = np.sqrt(252) * sector_returns.mean() / sector_returns.std() if sector_returns.std() > 0 else 0
    total_return = (1 + sector_returns).cumsum().iloc[-1]
    max_dd = (sector_returns.cumsum() / sector_returns.cumsum().cummax() - 1).min()
    
    return {
        'sharpe': sharpe,
        'total_return': total_return,
        'max_drawdown': max_dd
    }

# Test current parameters
result = backtest_sector_momentum(sector_data, vix_close, lookback=126, vix_threshold=25, leverage=1.5)
print("Current Parameters (lookback=126, vix=25, leverage=1.5):")
print(f"  Sharpe: {result['sharpe']:.3f}")
print(f"  Total Return: {result['total_return']*100:.1f}%")
print(f"  Max Drawdown: {result['max_drawdown']*100:.1f}%")

In [ ]:
# Grid search for optimal parameters
def grid_search_sector_momentum(sector_data, vix_data):
    results = []
    
    for lookback in [63, 90, 126, 180, 252]:  # 3, 4, 6, 9, 12 months
        for vix_thresh in [20, 25, 30, 35]:
            for leverage in [1.0, 1.25, 1.5, 2.0]:
                for top_n in [2, 3, 4, 5]:
                    result = backtest_sector_momentum(
                        sector_data, vix_data,
                        lookback=lookback,
                        vix_threshold=vix_thresh,
                        leverage=leverage,
                        top_n=top_n
                    )
                    
                    if result:
                        results.append({
                            'lookback': lookback,
                            'vix_threshold': vix_thresh,
                            'leverage': leverage,
                            'top_n': top_n,
                            'sharpe': result['sharpe'],
                            'total_return': result['total_return'],
                            'max_drawdown': result['max_drawdown']
                        })
    
    return pd.DataFrame(results)

print("Running grid search...")
print("(This may take a few minutes...)")
grid_results = grid_search_sector_momentum(sector_data, vix_close)
grid_results = grid_results.sort_values('sharpe', ascending=False)

print("\nTop 15 Parameter Combinations:")
print(grid_results.head(15).to_string())

## Research Findings & Recommendations

### Analysis Summary
Based on the grid search:

1. **Optimal Lookback**: The sweet spot for momentum calculation period
2. **VIX Threshold**: Balances risk reduction vs. opportunity cost
3. **Leverage**: Higher leverage increases returns but also drawdown
4. **Top N Sectors**: Diversification vs. concentration trade-off

### Recommended Parameters

```python
# In main.py or Alpha model
LOOKBACK_PERIOD = <VALUE>  # From grid search
VIX_THRESHOLD = <VALUE>  # From grid search
LEVERAGE = <VALUE>  # From grid search
TOP_N_SECTORS = <VALUE>  # From grid search
```